<a href="https://colab.research.google.com/github/vitamingyu/ml-for-Healthcare-Analytics/blob/main/diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 분석순서
# 딥러닝, 그리드 탐색을 사용한 당뇨병 진단
# 데이터셋 소개
# 케라스 모델 설계
# 사이킷런으로 그리드 탐색 실행하기
# 드롭아웃 규제로 과적합줄이기
# 최적의 초매개변수 찾기
# 최적의 초매개변수를 사용해 예측하기

In [2]:
# 딥러닝 알고리즘을 사용해 당뇨병 발생을 예측하는 방법과 그리드 탐색법을 사용해 알고리즘을 최적화하는 방법을 다룬다
import sys
import pandas as pd
import numpy as np
import sklearn
from tensorflow import keras

In [3]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['n_pregnant', 'glucose_concentration', 'blood_pressure (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 'BMI', 'pedigree_function', 'age', 'class']

df = pd.read_csv(url, names=names)
df.describe()

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [4]:
df[df['glucose_concentration']==0]
# 혈당이 0인 값(이상치) 필터링

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


In [5]:
columns=['glucose_concentration', 'blood_pressure (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 'BMI']
# n_prgenancy, age, class를 제외한 열
for col in columns:
  df[col].replace(0, np.nan, inplace=True)  # 0을 결측값으로 대체
df.describe()

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
df.dropna(inplace=True)
df.describe()

,n_pregnant,glucose_concentration,blood_pressure (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
# 데이터 프레임을 Numpy 배열로 변환하기 위해 .values속성을 사용
dataset = df.values
print(dataset.shape)  # (392, 9)

(392, 9)


In [8]:
# Numpy배열은 인덱싱하기에 편리
X = dataset[:, 0:8]
Y = dataset[:, 8].astype(int)  # 정수로 변환

print(X.shape)  # (392, 8)
print(Y.shape)  # (392,)

print(X[:5])  # 부동소수점
print(Y[:5])  # 정수

(392, 8)
(392,)
[[1.000e+00 8.900e+01 6.600e+01 2.300e+01 9.400e+01 2.810e+01 1.670e-01
  2.100e+01]
 [0.000e+00 1.370e+02 4.000e+01 3.500e+01 1.680e+02 4.310e+01 2.288e+00
  3.300e+01]
 [3.000e+00 7.800e+01 5.000e+01 3.200e+01 8.800e+01 3.100e+01 2.480e-01
  2.600e+01]
 [2.000e+00 1.970e+02 7.000e+01 4.500e+01 5.430e+02 3.050e+01 1.580e-01
  5.300e+01]
 [1.000e+00 1.890e+02 6.000e+01 2.300e+01 8.460e+02 3.010e+01 3.980e-01
  5.900e+01]]
[0 1 1 1 1]


In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)  # 스케일링을 위한 행렬이 내부에서 만들어진다
# fit 메서드는 스케일링을 위한 파라미터를 계산하는 역할,  훈련 데이터를 이용해 평균과 표준편차를 계산, 이 정보를 이용하여 나중에 데이터를 표준화합니다.

# 훈련 데이터의 변환(표준화)
X_standardized = scaler.transform(X)

data = pd.DataFrame(X_standardized)
data.describe()
# 정규화돼 평균 0, 표준편차 1을 가진다.
# 값들이 정규화돼 머신러닝 알고리즘이 모든 열에 대해 특별히 의존하거나 가중치를 부여하는 일 없이 균등한 값으로 처리할 수 있게 됨

,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-9.063045e-18,1.132881e-17,-4.531523e-16,1.087565e-16,1.064908e-16,1.631348e-16,1.812609e-17,1.110223e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


In [10]:
!pip install scikeras

from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from keras.optimizers import Adam

In [11]:
# 모델 정의 과정을 사용자 정의 함수로 만들 것임
# 앞으로 매개변수를 바꿔가면서 모델을 여러 번 초기화해 재사용할 것이기 때문에 함수로 정의

def create_model():
  # 케라스 모델 정의
  model = Sequential()
  model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
  model.add(Dense(4, input_dim = 8 ,kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  # 모델 컴파일
  adam = Adam(learning_rate = 0.01)
  model.compile(loss = 'bivary_crossentropy', optimizer=adam, metrics=['accuracy'])
  return model

model = create_model()
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 113 (452.00 Byte)
Trainable params: 113 (452.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [12]:
# 위 신경망의 모든 매개변수의 개수는 113개이다 Total params: 113
# 최적 배치 크기와 애포크를 정하기 위한 그리드 탐색
seed = 6
np.random.seed(seed)

# 모델 정의
def create_model():
  # 케라스 모델 생성
  model = Sequential()
  model.add(Dense(8, input_dim = 8,kernel_initializer='normal', activation='relu'))
  model.add(Dense(4, input_dim = 8,kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  # 모델 컴파일
  adam = Adam(lr = 0.01)
  model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
  return model

# 모델 생성
model = KerasClassifier(build_fn = create_model, verbose = 1)

# 그리드 탐색 매개변수 정의
batch_size = [10, 20, 40]
epochs = [10, 50, 100]

# 그리드 탐색 매개변수를 딕셔너리로 마들기
param_grid = dict(batch_size = batch_size, epochs = epochs)

# gridsearchcv빌드와 적합
grid = GridSearchCV(estimator = model, param_grid = param_grid,
                    cv = KFold(random_state=seed, shuffle=True),
                    verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# 결과 보고
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START batch_size=10, epochs=10....................................
Epoch 1/10
32/32 [==============================] - 3s 6ms/step - loss: 0.6897 - accuracy: 0.6550
Epoch 2/10
32/32 [==============================] - 0s 5ms/step - loss: 0.6810 - accuracy: 0.6805
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 0.6680 - accuracy: 0.6805
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 0.6486 - accuracy: 0.6805
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 0.6188 - accuracy: 0.6805
Epoch 6/10
32/32 [==============================] - 0s 3ms/step - loss: 0.5814 - accuracy: 0.6837
Epoch 7/10
32/32 [==============================] - 0s 3ms/step - loss: 0.5446 - accuracy: 0.7476
Epoch 8/10
32/32 [==============================] - 0s 4ms/step - loss: 0.5138 - accuracy: 0.8051
Epoch 9/10
32/32 [==============================] - 0s 3ms/step - loss: 0.4

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


[CV 1/5; 1/9] END .....batch_size=10, epochs=10;, score=0.772 total time=   4.6s
[CV 2/5; 1/9] START batch_size=10, epochs=10....................................
Epoch 1/10
32/32 [==============================] - 2s 8ms/step - loss: 0.6914 - accuracy: 0.6326
Epoch 2/10
32/32 [==============================] - 0s 9ms/step - loss: 0.6869 - accuracy: 0.6709
Epoch 3/10
32/32 [==============================] - 0s 10ms/step - loss: 0.6828 - accuracy: 0.6709
Epoch 4/10
32/32 [==============================] - 0s 6ms/step - loss: 0.6781 - accuracy: 0.6677
Epoch 5/10
32/32 [==============================] - 0s 10ms/step - loss: 0.6726 - accuracy: 0.6901
Epoch 6/10
32/32 [==============================] - 0s 6ms/step - loss: 0.6657 - accuracy: 0.7508
Epoch 7/10
32/32 [==============================] - 0s 5ms/step - loss: 0.6566 - accuracy: 0.7827
Epoch 8/10
32/32 [==============================] - 0s 5ms/step - loss: 0.6467 - accuracy: 0.7955
Epoch 9/10
32/32 [==============================] - 

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/100
8/8 [==============================] - 1s 6ms/step - loss: 0.6928 - accuracy: 0.6592
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: 0.6895 - accuracy: 0.6624
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - loss: 0.6849 - accuracy: 0.6624
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: 0.6795 - accuracy: 0.6624
Epoch 5/100
8/8 [==============================] - 0s 6ms/step - loss: 0.6732 - accuracy: 0.6624
Epoch 6/100
8/8 [==============================] - 0s 6ms/step - loss: 0.6659 - accuracy: 0.6624
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - loss: 0.6563 - accuracy: 0.6624
Epoch 8/100
8/8 [==============================] - 0s 5ms/step - loss: 0.6454 - accuracy: 0.6624
Epoch 9/100
8/8 [==============================] - 0s 6ms/step - loss: 0.6326 - accuracy: 0.6624
Epoch 10/100
8/8 [==============================] - 0s 5ms/step - loss: 0.6182 - accuracy: 0.6624
Epoch 11/100
8/8 [===========

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


[CV 4/5; 9/9] END ....batch_size=40, epochs=100;, score=0.821 total time=  11.2s
[CV 5/5; 9/9] START batch_size=40, epochs=100...................................
Epoch 1/100
8/8 [==============================] - 1s 4ms/step - loss: 0.6923 - accuracy: 0.5605
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: 0.6889 - accuracy: 0.6624
Epoch 3/100
8/8 [==============================] - 0s 4ms/step - loss: 0.6852 - accuracy: 0.6656
Epoch 4/100
8/8 [==============================] - 0s 4ms/step - loss: 0.6815 - accuracy: 0.6656
Epoch 5/100
8/8 [==============================] - 0s 4ms/step - loss: 0.6770 - accuracy: 0.6656
Epoch 6/100
8/8 [==============================] - 0s 4ms/step - loss: 0.6717 - accuracy: 0.6752
Epoch 7/100
8/8 [==============================] - 0s 4ms/step - loss: 0.6654 - accuracy: 0.6783
Epoch 8/100
8/8 [==============================] - 0s 4ms/step - loss: 0.6580 - accuracy: 0.7102
Epoch 9/100
8/8 [==============================] - 0s 4ms/step

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


[CV 5/5; 9/9] END ....batch_size=40, epochs=100;, score=0.769 total time=   5.9s
Epoch 1/10
40/40 [==============================] - 1s 3ms/step - loss: 0.6876 - accuracy: 0.6862
Epoch 2/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6708 - accuracy: 0.6811
Epoch 3/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6441 - accuracy: 0.7245
Epoch 4/10
40/40 [==============================] - 0s 3ms/step - loss: 0.6045 - accuracy: 0.7628
Epoch 5/10
40/40 [==============================] - 0s 3ms/step - loss: 0.5612 - accuracy: 0.7857
Epoch 6/10
40/40 [==============================] - 0s 3ms/step - loss: 0.5239 - accuracy: 0.7806
Epoch 7/10
40/40 [==============================] - 0s 4ms/step - loss: 0.4969 - accuracy: 0.7857
Epoch 8/10
40/40 [==============================] - 0s 3ms/step - loss: 0.4776 - accuracy: 0.7704
Epoch 9/10
40/40 [==============================] - 0s 3ms/step - loss: 0.4651 - accuracy: 0.7755
Epoch 10/10
40/40 [==================

In [13]:
#드롭아웃 정규화를 사용하여 과적합 줄임
# 최적 배치 크기와 에포크를 정하기 위한 그리드 탐색
# 필요한 패키지 임포트
from keras.layers import Dropout

# 랜덤 시드
seed = 6
np.random.seed(seed)

# 모델 정의
def create_model(learn_rate, dropout_rate): # 학습률과 드롭아웃 비율을 인자로
    # 카라스 모델 생성
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))        # 드롭아웃 레이어 추가
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))        # 드롭아웃 레이어 추가
    model.add(Dense(1, activation='sigmoid'))

    # 모델 컴파일
    adam = Adam(lr = learn_rate)            # 학습률에 대한 변수
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# 모델 생성
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=10, verbose=0, dropout_rate=0.0, learn_rate=0.001)

# 그리드 탐색 매개변수 정의
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# 그리드 탐색 매개변수를 딕셔너리로 변환
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# GridSearchCV 빌드와 적합
grid = GridSearchCV(estimator = model, param_grid = param_grid,
                    cv = KFold(random_state=seed, shuffle=True),
                    verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# 결과 보고
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START dropout_rate=0.0, learn_rate=0.001..........................


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


[CV 1/5; 1/9] END dropout_rate=0.0, learn_rate=0.001;, score=0.785 total time=   5.2s
[CV 2/5; 1/9] START dropout_rate=0.0, learn_rate=0.001..........................


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

[CV 3/5; 9/9] END dropout_rate=0.2, learn_rate=0.1;, score=0.769 total time=   6.3s
[CV 4/5; 9/9] START dropout_rate=0.2, learn_rate=0.1............................


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


[CV 4/5; 9/9] END dropout_rate=0.2, learn_rate=0.1;, score=0.821 total time=   5.4s
[CV 5/5; 9/9] START dropout_rate=0.2, learn_rate=0.1............................


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


[CV 5/5; 9/9] END dropout_rate=0.2, learn_rate=0.1;, score=0.782 total time=  11.0s
Best: 0.7756572541382667, using {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.7731256085686464 (0.041130453132937395) with: {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.767964946445959 (0.0457950134567102) with: {'dropout_rate': 0.0, 'learn_rate': 0.01}
0.7755923401493021 (0.046105660481134174) with: {'dropout_rate': 0.0, 'learn_rate': 0.1}
0.7654333008763388 (0.04660824407659851) with: {'dropout_rate': 0.1, 'learn_rate': 0.001}
0.7756572541382667 (0.03974008744113517) with: {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.7603700097370985 (0.04562853184270596) with: {'dropout_rate': 0.1, 'learn_rate': 0.1}
0.7604349237260629 (0.046602457512243664) with: {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.7655306718597858 (0.05079186759912183) with: {'dropout_rate': 0.2, 'learn_rate': 0.01}
0.7680298604349238 (0.0514898889832062) with: {'dropout_rate': 0.2, 'learn_rate': 0.1}


In [14]:
# 최적 초매개변수 찾기
# activation, init 그리드 탐색

# 랜덤 시드 지정
seed = 6
np.random.seed(seed)

# 모델 정의
def create_model(activation, init):  # 인자 지정
    # 모델 생성
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(4, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(1, activation='sigmoid'))

    # 모델 컴파일
    adam = Adam(lr = 0.001)   # 학습률을 하드코딩함.
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# 모델 생성
model = KerasClassifier(build_fn = create_model, epochs = 100,
                        batch_size = 20, verbose = 0)

# 그리드 탐색 매개변수 정의
activation = ['softmax', 'relu', 'tanh', 'linear']
init = ['uniform', 'normal', 'zero']

# 그리드 탐색 매개변수를 딕셔너리로 변환
param_grid = dict(activation = activation, init = init)

# GridSearchCV 빌드와 적합
grid = GridSearchCV(estimator = model, param_grid = param_grid,
                    cv = KFold(random_state=seed, shuffle=True), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# 결과 보고
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START activation=softmax, init=uniform...........................


ValueError: Invalid parameter activation for estimator KerasClassifier.
This issue can likely be resolved by setting this parameter in the KerasClassifier constructor:
`KerasClassifier(activation=softmax)`
Check the list of available parameters with `estimator.get_params().keys()`

In [ ]:
# 뉴런의 개수 최적화
# 히든 레이어의 뉴런의 최적 개수를 찾기 위한 그리드 탐색

# 랜덤 시드 설정
seed = 6
np.random.seed(seed)

# 모델 정의
def create_model(neuron1, neuron2):
    # 케라스 모델 생성
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 8,
                    kernel_initializer= 'uniform',
                    activation= 'linear'))
    model.add(Dense(neuron2, input_dim = neuron1,
                    kernel_initializer= 'uniform',
                    activation= 'linear'))
    model.add(Dense(1, activation='sigmoid'))

    # 모델 컴파일
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy',
                  optimizer = adam,
                  metrics = ['accuracy'])
    return model

# 모델 생성
model = KerasClassifier(build_fn = create_model,
                        epochs = 100, batch_size = 20, verbose = 0)

# 그리드 탐색 매개변수 설정
neuron1 = [4, 8, 16]
neuron2 = [2, 4, 8]

# 그리드 탐색 매개변수를 딕셔너리로 변화
param_grid = dict(neuron1 = neuron1, neuron2 = neuron2)

# GridSearchCV 빌드와 적합
grid = GridSearchCV(estimator = model,
                    param_grid = param_grid,
                    cv = KFold(random_state=seed, shuffle=True),
                    refit = True,
                    verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# 결과 보고
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

In [ ]:
# 최적의 초매개변수를 사용하여 예측하기
# 다음 코드와 같이 predict() 함수를 사용하여 데이터셋의 모든 사례에 대하여 당뇨병이 발생할지를 예측해 보자.

# 최적 초매개변수를 가지고 예측값 생성
import numpy as np
y_pred = grid.predict(X_standardized)

In [ ]:
print(y_pred.shape)
print(y_pred[:5])
# 예측값의 형태

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))

In [ ]:
example = df.iloc[1]
print(example)

In [ ]:
prediction = grid.predict(X_standardized[1].reshape(1, -1))
print(prediction)

In [ ]:
# 요약
# 이 장에서 우리는 케라스를 사용하여 딥러닝 신경망을 만들고, 사이킷런 그리드 탐색법을 사용하여 최적의 초매개변수를 찾아보았다.
# 그리고 초매개변수를 조정하여 신경망을 최적화시키는 방법을 배웠다.우리의 모델이 비교적 성공적이라고 생각할 수 있다.
# 새로운 데이터로 또는 새로운 데이터로 다른 문제를 접했을 때도 이와 비슷한 과정을 통해서 문제를 해결할 수 있다.
# 이 장에 우리는 딥러닝, 초매개변수 최적화 방법을 배웠고, 이것을 신경망에 적용하여 당뇨병 환자 데이터에서 당뇨병의 발명을 예측하는 데 응용할 수 있었다.

# 다음 장에서는 머신러닝을 사용하여 DNA를 분류하는 방법에 대해서 살펴볼 것이다.